# Turbulence Simulation with Julia: Kolmogorov Flow

**Author**: Mingyu Jeon   
**Date**: 2025-08-07

---

Kolmogorov flow is a shear flow with spatially periodic forcing {cite:p}`Chatterjee2020`. It is a well-known test-bed for simulating turbulence {cite:p}`fylladitakis2018kolmogorov`.

The Navier-Stokes equations for an incompressible fluid are

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} = - \frac{1}{\rho}\nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f}
$$

$$
\nabla\cdot \mathbf{u} = 0
$$

where  $\rho$ is the density, $\mathbf{u}$ is the velocity, $p$ is the pressure, $\nu$ is the kinematic viscosity, and $\mathbf{f}$ is the forcing.

We assume that density $\rho$ is constant.

When we consider a 2D Kolmogorov flow in a periodic and rectangular domain in the xy-plane, the forcing $\mathbf{f}$ is given by

$$
\mathbf{f}(\mathbf{x}) = c \sin(l \pi y) \hat{\mathbf{x}}
$$

where forcing scale $c$ and forcing wavenumber $l$ are constants.

In this post, we will simulate the 2D Kolmogorov flow with [Julia](https://julialang.org/) using the ["Stable Fluids"](https://www.josstam.com/_files/ugd/cf1fd6_898fe9b63df946689101b8d074f8efba.pdf) algorithm based on this video: [Writing a Turbulence Simulation in Julia](https://www.youtube.com/watch?v=26vdFV1EXKk).

## Stable Fluids algorithm using FFT

The explanation of the Stable Fluids algorithm using FFT is here: [Introduction to “Stable Fluids”](https://khu-sciml.netlify.app/posts/intro_stable_fluids). 

Here I just summarize the algorithm:

- Initialize: $\mathbf{w}_0(\mathbf{x}) = \mathbf{u}(\mathbf{x}, t)$
- Add Force: $\mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x})$
- Advect: $\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x}))$
- Fourier Transform: $\hat{\mathbf{w}}_2(\mathbf{k}) = \mathcal{F}\{ \mathbf{w}_2(\mathbf{x}) \}$
- Diffuse: $\hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w}}_2(\mathbf{k}) / (1 + \nu \Delta t \; k^2)$
- Project: $\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - \mathbf{k} (\mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k}))/k^2$
- Inverse Fourier Transform: $\mathbf{w}_4(\mathbf{x}) = \mathcal{F}^{-1}\{ \hat{\mathbf{w}}_4(\mathbf{k}) \}$
- Update: $\mathbf{u}(\mathbf{x}, t + \Delta t) = \mathbf{w}_4(\mathbf{x})$

### Setup Parameters

First, we set up spatial domain parameters: aspect ratio & number of points in y-direction.

In [ ]:
ASPECT_RATIO = 16/9     # x-size / y-size
N_POINTS_Y = 360;       # number of points in y-direction

Next, we set up temporal domain parameters: time step length $\Delta t$ & number of time steps.

In [ ]:
TIME_STEP_LENGTH = 0.001   # time step length
N_TIME_STEPS = 750;        # number of time steps

Finally, we set up physical parameters: kinematic viscosity $\nu$, forcing scale $c$, and forcing wavenumber $l$.

In [ ]:
KINEMATIC_VISCOSITY = 1.0 / 1000.0   # kinematic viscosity
FORCING_SCALE = 100.0                # forcing scale
FORCING_WAVENUMBER = 8;              # forcing wavenumber

### Prepare Spatial Domain

We construct the coordinate grids $(x, y)$ in the spatial domain.

In [ ]:
n_points_x = Int(ASPECT_RATIO * N_POINTS_Y)           # number of points in x-direction (integer)

640

In [ ]:
x_extent = n_points_x / N_POINTS_Y - 1.0e-7           # length of x-extent
x_interval = range(0.0, x_extent, length=n_points_x)  # range of x-coordinates

0.0:0.0027821246913580246:1.7777776777777776

In [ ]:
y_interval = range(0.0, 1.0, length=N_POINTS_Y)       # range of y-coordinates

0.0:0.002785515320334262:1.0

In [ ]:
coordinates_x = [x for x in x_interval, y in y_interval]  # x-coordinates in 2D array (Nx x Ny)

640×360 Matrix{Float64}:
 0.0         0.0         0.0         …  0.0         0.0         0.0
 0.00278212  0.00278212  0.00278212     0.00278212  0.00278212  0.00278212
 0.00556425  0.00556425  0.00556425     0.00556425  0.00556425  0.00556425
 0.00834637  0.00834637  0.00834637     0.00834637  0.00834637  0.00834637
 0.0111285   0.0111285   0.0111285      0.0111285   0.0111285   0.0111285
 0.0139106   0.0139106   0.0139106   …  0.0139106   0.0139106   0.0139106
 0.0166927   0.0166927   0.0166927      0.0166927   0.0166927   0.0166927
 0.0194749   0.0194749   0.0194749      0.0194749   0.0194749   0.0194749
 0.022257    0.022257    0.022257       0.022257    0.022257    0.022257
 0.0250391   0.0250391   0.0250391      0.0250391   0.0250391   0.0250391
 ⋮                                   ⋱                          
 1.75552     1.75552     1.75552        1.75552     1.75552     1.75552
 1.7583      1.7583      1.7583         1.7583      1.7583      1.7583
 1.76108     1.76108     1.7610

In [ ]:
coordinates_y = [y for x in x_interval, y in y_interval]  # y-coordinates in 2D array (Nx x Ny)

640×360 Matrix{Float64}:
 0.0  0.00278552  0.00557103  …  0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103  …  0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 ⋮                            ⋱                                
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.991643  0.994429  0.997214  1.0
 0.0  0.00278552  0.00557103     0.99164

### Prepare Fourier Domain

We construct the wavenumber grids $(k_x, k_y)$ in the Fourier domain.

In [ ]:
using FFTW

Since we use FFT for 2D domain and physical quantities are real numbers, we can apply efficient `rfft` in x-direction and `fft` in y-direction. The reason of `fft` in y-direction is that the the result of `rfft` in x-direction is complex numbers, so we need to use `fft` in y-direction to handle complex numbers.

The following code use `rfftfreq` in x-direction and `fftfreq` in y-direction to construct the wavenumber grids.

In [ ]:
wavenumbers_1d_x = rfftfreq(n_points_x) .* n_points_x

321-element Frequencies{Float64}:
   0.0
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
   ⋮
 312.0
 313.0
 314.0
 315.0
 316.0
 317.0
 318.0
 319.0
 320.0

In [ ]:
wavenumbers_1d_y = fftfreq(N_POINTS_Y) .* N_POINTS_Y

360-element Frequencies{Float64}:
  0.0
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
  ⋮
 -9.0
 -8.0
 -7.0
 -6.0
 -5.0
 -4.0
 -3.0
 -2.0
 -1.0

In [ ]:
n_fft_points_x = length(wavenumbers_1d_x)   # number of wavenumber grids in x-direction
n_fft_points_y = length(wavenumbers_1d_y);  # number of wavenumber grids in y-direction

In [ ]:
wavenumbers_x = [k_x for k_x in wavenumbers_1d_x, k_y in wavenumbers_1d_y]

321×360 Matrix{Float64}:
   0.0    0.0    0.0    0.0    0.0  …    0.0    0.0    0.0    0.0    0.0
   1.0    1.0    1.0    1.0    1.0       1.0    1.0    1.0    1.0    1.0
   2.0    2.0    2.0    2.0    2.0       2.0    2.0    2.0    2.0    2.0
   3.0    3.0    3.0    3.0    3.0       3.0    3.0    3.0    3.0    3.0
   4.0    4.0    4.0    4.0    4.0       4.0    4.0    4.0    4.0    4.0
   5.0    5.0    5.0    5.0    5.0  …    5.0    5.0    5.0    5.0    5.0
   6.0    6.0    6.0    6.0    6.0       6.0    6.0    6.0    6.0    6.0
   7.0    7.0    7.0    7.0    7.0       7.0    7.0    7.0    7.0    7.0
   8.0    8.0    8.0    8.0    8.0       8.0    8.0    8.0    8.0    8.0
   9.0    9.0    9.0    9.0    9.0       9.0    9.0    9.0    9.0    9.0
   ⋮                                ⋱    ⋮                         
 312.0  312.0  312.0  312.0  312.0     312.0  312.0  312.0  312.0  312.0
 313.0  313.0  313.0  313.0  313.0     313.0  313.0  313.0  313.0  313.0
 314.0  314.0  314.0  314.0  31

In [ ]:
wavenumbers_y = [k_y for k_x in wavenumbers_1d_x, k_y in wavenumbers_1d_y]

321×360 Matrix{Float64}:
 0.0  1.0  2.0  3.0  4.0  5.0  6.0  …  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0  …  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 ⋮                        ⋮         ⋱         ⋮                      
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3.0  4.0  5.0  6.0     -6.0  -5.0  -4.0  -3.0  -2.0  -1.0
 0.0  1.0  2.0  3

In [ ]:
using LinearAlgebra

We calculate the norm of the wavenumber vectors $k=|\mathbf{k}|$ in the Fourier domain.

In [ ]:
wavenumbers_norm = [norm([k_x, k_y]) for k_x in wavenumbers_1d_x, k_y in wavenumbers_1d_y]

321×360 Matrix{Float64}:
   0.0    1.0        2.0        3.0      …    3.0        2.0        1.0
   1.0    1.41421    2.23607    3.16228       3.16228    2.23607    1.41421
   2.0    2.23607    2.82843    3.60555       3.60555    2.82843    2.23607
   3.0    3.16228    3.60555    4.24264       4.24264    3.60555    3.16228
   4.0    4.12311    4.47214    5.0           5.0        4.47214    4.12311
   5.0    5.09902    5.38516    5.83095  …    5.83095    5.38516    5.09902
   6.0    6.08276    6.32456    6.7082        6.7082     6.32456    6.08276
   7.0    7.07107    7.28011    7.61577       7.61577    7.28011    7.07107
   8.0    8.06226    8.24621    8.544         8.544      8.24621    8.06226
   9.0    9.05539    9.21954    9.48683       9.48683    9.21954    9.05539
   ⋮                                     ⋱                        
 312.0  312.002    312.006    312.014       312.014    312.006    312.002
 313.0  313.002    313.006    313.014       313.014    313.006    313.002
 314

We then caculate $\text{decay} = 1 / (1 + \nu \Delta t \;k^2)$ to use it in the "diffuse" step.

In [ ]:
decay = 1 ./ (1 .+ KINEMATIC_VISCOSITY .* TIME_STEP_LENGTH .*  wavenumbers_norm.^2)

321×360 Matrix{Float64}:
 1.0       0.999999  0.999996  0.999991  …  0.999991  0.999996  0.999999
 0.999999  0.999998  0.999995  0.99999      0.99999   0.999995  0.999998
 0.999996  0.999995  0.999992  0.999987     0.999987  0.999992  0.999995
 0.999991  0.99999   0.999987  0.999982     0.999982  0.999987  0.99999
 0.999984  0.999983  0.99998   0.999975     0.999975  0.99998   0.999983
 0.999975  0.999974  0.999971  0.999966  …  0.999966  0.999971  0.999974
 0.999964  0.999963  0.99996   0.999955     0.999955  0.99996   0.999963
 0.999951  0.99995   0.999947  0.999942     0.999942  0.999947  0.99995
 0.999936  0.999935  0.999932  0.999927     0.999927  0.999932  0.999935
 0.999919  0.999918  0.999915  0.99991      0.99991   0.999915  0.999918
 ⋮                                       ⋱                      
 0.911291  0.91129   0.911288  0.911284     0.911284  0.911288  0.91129
 0.910773  0.910772  0.910769  0.910765     0.910765  0.910769  0.910772
 0.910253  0.910252  0.910249  0.9102

We replace 0 with 1 for the wavenumber $k=0$ to avoid division by zero in the "project" step: $\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - \mathbf{k} (\mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k}))/k^2$

In [ ]:
wavenumbers_norm[iszero.(wavenumbers_norm)] .= 1.0;

We calculate the forcing $\mathbf{f}(\mathbf{x}) = c \sin(l \pi y) \hat{\mathbf{x}}$ to use it in the "add force" step: $\mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x})$

In [ ]:
force_x = FORCING_SCALE * sin.(FORCING_WAVENUMBER * pi * coordinates_y)

640×360 Matrix{Float64}:
 0.0  6.99505  13.9558  20.8482  27.6385  …  -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385  …  -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 ⋮                                        ⋱                      
 0.0  6.99505  13.9558  20.8482  27.6385     -13.9558  -6.99505  -9.79717e-14
 0.0  6.99505  13.9558  20.8482  27.6385     -13.95

### Preallocate Arrays

previous velocity $\mathbf{u}(\mathbf{x}, t)$

In [ ]:
velocity_x_prev = zeros(Float32, n_points_x, N_POINTS_Y)
velocity_y_prev = zeros(Float32, n_points_x, N_POINTS_Y);

next velocity $\mathbf{u}(\mathbf{x}, t + \Delta t)$

In [ ]:
velocity_x = zeros(Float32, n_points_x, N_POINTS_Y)
velocity_y = zeros(Float32, n_points_x, N_POINTS_Y);

velocity in Fourier domain $\hat{\mathbf{u}}(\mathbf{k})$

In [ ]:
velocity_x_fft = zeros(Complex{Float32}, n_fft_points_x, n_fft_points_y)
velocity_y_fft = zeros(Complex{Float32}, n_fft_points_x, n_fft_points_y);

pseudo-pressure in Fourier domain $q(\mathbf{k}) = (\mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k}))/k^2$

Then, the "project" step is $\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - q(\mathbf{k})\mathbf{k} $

In [ ]:
pseudo_pressure_fft = zeros(Complex{Float32}, n_fft_points_x, n_fft_points_y);

backtraced position $\mathbf{p}(\mathbf{x},-\Delta t)$ 

In [ ]:
backtraced_coordinates_x = zeros(Float32, n_points_x, N_POINTS_Y)
backtraced_coordinates_y = zeros(Float32, n_points_x, N_POINTS_Y);

The backtraced position can be calculated as

$$
\mathbf{p}(\mathbf{x},-\Delta t) = \mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x})
$$

and it is used in the "advect" step

$$
\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x}))
$$

Since the backtraced position does not necessarily lie on the grid points, we need an interpolator for $\mathbf{w}_1(\mathbf{x})$ at those positions.

In [ ]:
using Interpolations

In [ ]:
interpolator = interpolate(
    (x_interval, y_interval), # (x-coordinates, y-coordinates) ranges
    velocity_x,               # (Nx, Ny) zero array for an initialization of the interpolator
    Gridded(Linear()),        # Bi-linear interpolation
)

640×360 interpolate((0.0:0.0027821246913580246:1.7777776777777776,0.0:0.002785515320334262:1.0), ::Matrix{Float32}, Gridded(Linear())) with element type Float32:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮   

For visualization, we will calculate the curl of the velocity $\nabla \times \mathbf{u}(\mathbf{x})$.

The curl of the velocity in Fourier domain is $i\mathbf{k} \times \hat{\mathbf{u}}(\mathbf{k})$.

For 2D velocity $\hat{\mathbf{u}} = (\hat{u}_x, \hat{u}_y)$, the curl in Fourier domain is given by

$$
i (k_x \hat{\mathbf{x}} + k_y \hat{\mathbf{y}}) \times (\hat{u}_x \hat{\mathbf{x}} + \hat{u}_y \hat{\mathbf{y}}) = i (k_x \hat{u}_y - k_y \hat{u}_x) \hat{\mathbf{z}}
$$

Therefore,

$$
\nabla \times \mathbf{u}(\mathbf{x}) = \mathcal{F}^{-1}\{ i k_x \hat{u}_y - ik_y \hat{u}_x \} \hat{\mathbf{z}}
$$

In [ ]:
curl_fft = zeros(Complex{Float32}, n_fft_points_x, n_fft_points_y)
curl = zeros(Float32, n_points_x, N_POINTS_Y);

### Main Loop

The next cell is the main simulation loop.

In [ ]:
using ProgressMeter # for @showprogress
using Printf        # for @sprintf
using Statistics    # for mean
using Plots         # visualization
theme(:dark)        # dark theme for Plots

mkpath("images")  # Create a directory to save the images

@showprogress "Timestepping ..." for iter in 1:N_TIME_STEPS
    # (1) Add Force
    velocity_x_prev .+= TIME_STEP_LENGTH * force_x

    # (2) Self-Advection by backtracing and interpolation
    backtraced_coordinates_x .= mod1.(
        coordinates_x - TIME_STEP_LENGTH * velocity_x_prev,
        x_extent,
    )  # Here, mod1 is used to ensure that the coordinates wrap around the domain (periodic boundary conditions)
    backtraced_coordinates_y .= mod1.(
        coordinates_y - TIME_STEP_LENGTH * velocity_y_prev,
        1.0,  # y-extent
    )  # Again, mod1 is used for periodic boundary conditions

    interpolator.coefs .= velocity_x_prev  # We create a new interpolator with the current velocity_x_prev
    velocity_x .= interpolator.(
        backtraced_coordinates_x, 
        backtraced_coordinates_y
    )  # Get the velocity_x at the backtraced coordinates (self-advection)
    interpolator.coefs .= velocity_y_prev  # We create a new interpolator with the current velocity_y_prev
    velocity_y .= interpolator.(
        backtraced_coordinates_x, 
        backtraced_coordinates_y
    )  # Get the velocity_y at the backtraced coordinates (self-advection)

    # Subtle Point: Stabilize by subtracting the mean velocities
    # It avoids the drift of the flow
    # We always in the fluid's reference frame so that we only focus on the vortex motions
    velocity_x .-= mean(vec(velocity_x))
    velocity_y .-= mean(vec(velocity_y))
    
    # Fourier Transform: `rfft`
    velocity_x_fft = rfft(velocity_x)
    velocity_y_fft = rfft(velocity_y)

    # (3) Diffuse
    velocity_x_fft .*= decay
    velocity_y_fft .*= decay

    # (4) Project to the divergence-free space
    pseudo_pressure_fft = (
        velocity_x_fft .* wavenumbers_x
        +
        velocity_y_fft .* wavenumbers_y
    ) 
    pseudo_pressure_fft ./= wavenumbers_norm.^2

    velocity_x_fft -= pseudo_pressure_fft .* wavenumbers_x
    velocity_y_fft -= pseudo_pressure_fft .* wavenumbers_y

    # Inverse Fourier Transform: `irfft`
    velocity_x = irfft(velocity_x_fft, n_points_x)
    velocity_y = irfft(velocity_y_fft, n_points_x)

    # Subtle Point: Stabilize by subtracting the mean velocities
    velocity_x .-= mean(vec(velocity_x))
    velocity_y .-= mean(vec(velocity_y))

    # (5) Advance in time
    velocity_x_prev = velocity_x
    velocity_y_prev = velocity_y

    # Visualize
    curl_fft .= im .* wavenumbers_x .* velocity_y_fft - im .* wavenumbers_y .* velocity_x_fft
    curl .= irfft(curl_fft, n_points_x)
    curl = sign.(curl) .* sqrt.(abs.(curl) ./ quantile(vec(curl), 0.8)) # Normalize the curl for visualization

    heatmap(
        x_interval, 
        y_interval,
        curl', # Transpose for correct orientation
        c = :seaborn_icefire_gradient,
        size = (1920, 1080),
        clim = (-5.0, 5.0),
        axis = false,
        showaxis = false,
        legend = :none,
        ticks = false,
        margin = 0.0Plots.mm,
        annotations = (
            0.06, 
            1.0 - 0.06,
            Plots.text(
                "$(@sprintf("iter: %05d", iter))",
                pointsize = 40,
                color = :white,
                halign = :left,
            )
        )
    )
    savefig("images/kolmogorov_$(@sprintf("%05d", iter)).png")
    
end 

Timestepping ... 100%|███████████████████████████████████| Time: 0:03:47


## Simulation Result

The following [FFmpeg](https://ffmpeg.org/) command can be used to create a video.

```bash
ffmpeg -f images -framerate 24 -i kolmogorov_%05d.png kolmogorov_animation.mp4
```

```{raw} html
<div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;">
  <iframe 
    src="https://www.youtube.com/embed/m_Lj1vyoNk0" 
    style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"
    frameborder="0" 
    allowfullscreen>
  </iframe>
</div>
```

```{bibliography}
```